In [2]:
api_key= "ea0518bbe4e05ba4b9685668b348a1d7"

api_private_key = "6704343727a4d01b047aec1033b8478f540932c9"

url_base = "http://gateway.marvel.com"

endpoint_lists = "/v1/public/characters"


In [3]:
import requests

In [4]:
# Provide a list of 30 Marvel characters

url_list = url_base + endpoint_lists

import time

ts = 1000

ts_str = str(int(ts))

import hashlib

hash1 = str(ts_str)+api_private_key+api_key

hash2= hashlib.md5(hash1.encode())

hash3= hash2.hexdigest()


body = {'limit':30, 'ts':ts_str, 'apikey': api_key, 'hash':hash3}

characters = requests.get(url=url_list, params= body).json()

<b>1. Provide a list of 30 Marvel characters</b>

In [5]:
list_characters=[]
for i in characters['data']['results']:
    for j in i:
        if j == 'name':
            list_characters.append(i[j])

In [6]:
print(list_characters)

['3-D Man', 'A-Bomb (HAS)', 'A.I.M.', 'Aaron Stack', 'Abomination (Emil Blonsky)', 'Abomination (Ultimate)', 'Absorbing Man', 'Abyss', 'Abyss (Age of Apocalypse)', 'Adam Destine', 'Adam Warlock', 'Aegis (Trey Rollins)', 'Aero (Aero)', 'Agatha Harkness', 'Agent Brand', 'Agent X (Nijo)', 'Agent Zero', 'Agents of Atlas', 'Aginar', 'Air-Walker (Gabriel Lan)', 'Ajak', 'Ajaxis', 'Akemi', 'Alain', 'Albert Cleary', 'Albion', 'Alex Power', 'Alex Wilder', 'Alexa Mendez', 'Alexander Pierce']


<b>2. Retrieve the Ids for all the characters in your list (in str form)</b>

In [7]:
list_ids=[]
for i in characters['data']['results']:
    for j in i:
        if j == 'id':
            list_ids.append(str(i[j]))

In [42]:
print(list_ids)

['1011334', '1017100', '1009144', '1010699', '1009146', '1016823', '1009148', '1009149', '1010903', '1011266', '1010354', '1010846', '1017851', '1012717', '1011297', '1011031', '1009150', '1011198', '1011175', '1011136', '1011176', '1010870', '1011194', '1011170', '1009240', '1011120', '1010836', '1010755', '1011214', '1009497']


30

<b>3. Retrieve the total number of Events available for all the characters in your list (in integer form)</b>

In [9]:
endpoint_events = "/v1/public/characters/{characterId}/events".format(characterId=1011334)

In [85]:
body_events= { 'ts':ts_str,'apikey': api_key,'hash':hash3}
list_events = []
for i in list_ids:
    endpoint_events = "/v1/public/characters/{characterId}/events".format(characterId=int(i))
    url_final_events = url_base + endpoint_events
    c= requests.get(url=url_final_events, params= body_events).json()
    if c['data']['count'] == 0:
        list_events.append(None)
    else:
        list_events.append(c['data']['count'])

In [86]:
list_events

[1,
 None,
 None,
 None,
 1,
 None,
 4,
 1,
 1,
 None,
 8,
 None,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 1,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

<b>4. Retrieve the total number of Series available for all the characters in your list  (in integer form)</b>

In [87]:
body_series= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous
list_series = []
for i in list_ids:
    endpoint_events = "/v1/public/characters/{characterId}/series".format(characterId=int(i))
    url_final_series = url_base + endpoint_events
    c= requests.get(url=url_final_series, params= body_series).json()
    if c['data']['count'] == 0:
        list_series.append(None)
    else:
        list_series.append(c['data']['count'])            

In [88]:
list_series

[3,
 2,
 20,
 3,
 20,
 2,
 20,
 3,
 3,
 None,
 20,
 None,
 5,
 10,
 7,
 3,
 10,
 13,
 None,
 3,
 1,
 None,
 None,
 None,
 None,
 1,
 6,
 4,
 None,
 1]

<b>5. Retrieve the total number of Comics available for all the characters in your list (in integer form)</b>

In [89]:
body_comics= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous
list_comics = []
for i in list_ids:
    endpoint_comics = "/v1/public/characters/{characterId}/comics".format(characterId=int(i))
    url_final_comics = url_base + endpoint_comics
    c= requests.get(url=url_final_comics, params= body_comics).json()
    if c['data']['count'] == 0:
        list_comics.append(None) 
    else:
        list_comics.append(c['data']['count']) 

In [90]:
list_comics

[12,
 4,
 20,
 14,
 20,
 2,
 20,
 8,
 3,
 None,
 20,
 None,
 20,
 20,
 20,
 18,
 20,
 20,
 None,
 4,
 4,
 None,
 None,
 None,
 None,
 1,
 18,
 9,
 None,
 1]

<b> 6. Retrieve the Price of the most expensive comic that the character was featured in or all the characters in your list (in float form and USD)</b>

In [80]:
#Needs to be completed
body_prices= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous
dic_prices= {}
for i in list_ids:
    endpoint_comics = "/v1/public/characters/{characterId}/comics".format(characterId=int(i))
    url_final_comics = url_base + endpoint_comics
    c= requests.get(url=url_final_comics, params= body_comics).json()
    if len(c['data']['results'])==0:
        x= None
        dic_prices.setdefault(i, [])
        dic_prices[i].append(x)
    else:
        for j in c['data']['results']:#for each comic in the list
            for k in j['prices']:
                x= k['price']
                dic_prices.setdefault(i, [])
                dic_prices[i].append(x)
print (dic_prices) # The ids not appearing in the list mean no comics found

{'1011334': [2.99, 1.99, 2.99, 1.99, 2.99, 2.99, 1.99, 2.99, 1.99, 2.99, 1.99, 2.99, 1.99, 2.99, 0, 1.99, 0, 0, 0], '1017100': [0.2, 0, 2.99, 1.99, 2.99, 1.99, 2.99, 1.99], '1009144': [0, 0, 39.99, 125, 0, 39.99, 0, 0, 0, 0, 3.99, 1.99, 3.99, 1.99, 3.99, 1.99, 3.99, 1.99, 3.99, 1.99, 2.99, 14.99, 3.99, 1.99, 3.99, 0], '1010699': [2.99, 1.99, 2.99, 1.99, 2.99, 2.99, 1.99, 2.99, 1.99, 2.99, 1.99, 2.99, 0, 1.99, 0, 1.99, 0, 1.99, 0, 1.99, 0, 1.99, 0, 1.99, 0, 1.99], '1009146': [3.99, 3.99, 75, 39.99, 34.99, 3.99, 1.99, 3.99, 1.99, 3.99, 2.99, 1.99, 9.99, 19.99, 2.99, 1.99, 9.99, 49.99, 2.99, 9.99, 9.99, 9.99, 0, 0], '1016823': [3.99, 1.99, 0, 1.99], '1009148': [3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 15.99, 3.99, 3.99, 75, 17.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 1.99], '1009149': [9.99, 0, 1.99, 0, 1.99, 0, 1.99, 0, 1.99, 0, 1.99, 0, 1.99, 3.95, 1.99], '1010903': [9.99, 1.95, 1.99, 3

In [81]:
list_max_prices=[] #not optimal I think
for x in dic_prices:
    if dic_prices[x]==None:
        list_max_prices.append(None)
    else:
        i= max(dic_prices[x])
        list_max_prices.append(i)

In [82]:
list_max_prices #Not sure it is in USD

[2.99,
 2.99,
 125,
 2.99,
 75,
 3.99,
 75,
 9.99,
 9.99,
 None,
 125,
 None,
 4.99,
 49.99,
 15.99,
 9.99,
 3.95,
 39.99,
 None,
 9.99,
 2.99,
 None,
 None,
 None,
 None,
 2.99,
 6.99,
 3.99,
 None,
 2.99]

<b>7. Store the data above in a pandas DataFrame</b>

In [51]:
import pandas as pd

In [91]:
df = pd.DataFrame({'Character Name':list_characters, 'Character ID':list_ids,'Total Available Events':list_events,'Total Available Series':list_series,'Total Available Comics':list_comics,'Price of the Most Expensive Comic':list_max_prices })

In [92]:
df

,Character Name,Character ID,Total Available Events,Total Available Series,Total Available Comics,Price of the Most Expensive Comic
0,3-D Man,1011334,1.0,3.0,12.0,2.99
1,A-Bomb (HAS),1017100,NaN,2.0,4.0,2.99
2,A.I.M.,1009144,NaN,20.0,20.0,125.00
3,Aaron Stack,1010699,NaN,3.0,14.0,2.99
4,Abomination (Emil Blonsky),1009146,1.0,20.0,20.0,75.00
5,Abomination (Ultimate),1016823,NaN,2.0,2.0,3.99
6,Absorbing Man,1009148,4.0,20.0,20.0,75.00
7,Abyss,1009149,1.0,3.0,8.0,9.99
8,Abyss (Age of Apocalypse),1010903,1.0,3.0,3.0,9.99
9,Adam Destine,1011266,NaN,NaN,NaN,NaN


In [94]:
df.to_csv(path_or_buf="data.csv")